# Notebook 2:  Data Cleaning 

In this notebook we merge our raw data tables from Notebook 1 and clean the player salaries.
1.  **Merging**: We update our 2020 free agent data with the 2021 next-season salaries, and also merge current-season (2021-22) stats and salaries together.  This gives us two new dataframes:  df2020_merged and df2021_merged.
2.  **Cleaning salaries**: We convert salaries into numerical form.  We also normalize salaries from all years to the 2021 salary cap for consistency.

## Imports

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup, Comment
import requests
import lxml
import unicodedata
import json, pickle

## Utility Functions

In [2]:
def strip_accents_and_punctuation(text):
    '''Normalize player name spellings'''
    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass
    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")
    return str(text).replace('.','').replace(',','').replace("'",'')

# (1) Merge Scraped Data

Merging 2021 data with current data

In [22]:
df2022 = pd.read_csv('./data/df2022_raw.csv')
df2023salaries = pd.read_csv('./data/dfcurrentsalaries.csv')
df2023stats = pd.read_csv('./data/dfcurrentstats.csv')
df2023urls = pd.read_csv('./data/dfplayer_to_url.csv')
df2021_merged = pd.read_csv('./data/df2021_merged.csv')

## a. Update df2020 with 2021 salaries

In [24]:
df2022_merged = pd.merge(df2021_merged, df2023salaries, left_on="Name", right_on="Name", how="left")
df2022_merged = df2021_merged.drop(columns=['Salary'])
df2022_merged = df2021_merged.rename(columns = {'CurrentSalary':'Salary'})
df2022_merged = df2021_merged.set_index('NameYear')
df2022_merged.head(3)

,Name,Pos,Age,TeamOrTot,G,GS,MP,FG,FGA,FG%,...,TOV%,TRB%,TS%,USG%,VORP,WS,WS/48,Salary,ID,Team
NameYear,,,,,,,,,,,,,,,,,,,,,
Precious_Achiuwa_2021,Precious Achiuwa,C,22,TOR,73,28,23.6,3.6,8.3,.439,...,11.3,14.9,.503,18.5,-0.2,2.5,.070,"$2,711,280",/players/a/achiupr01.html,TOR
Steven_Adams_2021,Steven Adams,C,28,MEM,76,75,26.3,2.8,5.1,.547,...,19.6,19.9,.560,12.0,2.0,6.8,.163,"$17,073,171",/players/a/adamsst01.html,MEM
Bam_Adebayo_2021,Bam Adebayo,C,24,MIA,56,56,32.6,7.3,13.0,.557,...,14.4,17.5,.608,25.0,2.7,7.2,.188,"$28,103,550",/players/a/adebaba01.html,MIA


## b. Update df2022 with 2023 salaries and player urls

In [25]:
df2022_merged = pd.merge(df2023stats, df2023salaries, left_on="Player", right_on="Name", how="left")
df2022_merged = pd.merge(df2022_merged, df2023urls, left_on="Player", right_on="Name", how="left")

df2022_merged = df2022_merged.drop(columns=['Name_x', 'Name_y','Unnamed: 0','Unnamed: 0_x','Unnamed: 0_y','Rk'])
df2022_merged = df2022_merged.rename(columns = {'Player':'Name'})

df2022_merged.head(3)

,Name,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,TOV%,TRB%,TS%,USG%,VORP,WS,WS/48,CurrentSalary,CurrentTeam,ID
0,Precious Achiuwa,C,22,TOR,73,28,23.6,3.6,8.3,.439,...,11.3,14.9,.503,18.5,-0.2,2.5,.070,"$4,379,527",TOR,/players/a/achiupr01.html
1,Steven Adams,C,28,MEM,76,75,26.3,2.8,5.1,.547,...,19.6,19.9,.560,12.0,2.0,6.8,.163,"$12,600,000",MEM,/players/a/adamsst01.html
2,Bam Adebayo,C,24,MIA,56,56,32.6,7.3,13.0,.557,...,14.4,17.5,.608,25.0,2.7,7.2,.188,"$32,600,060",MIA,/players/a/adebaba01.html


## c. Update df2022 with current team and total stats

This is a little bit tricky because some players played for multiple teams.  As an example, Nickeil Alexander-Walker switched teams mid-season and thus has three data rows:

* Name    /   Team
* Nickeil Alexander-Walker  /    TOT
* Nickeil Alexander-Walker  /    NOP
* Nickeil Alexander-Walker  /    UTA

We want to keep his **season stats** from the first row (TOT) but also indicate his **current team** from the last row (UTA).  The same pattern holds for all players who played for multiple teams.

In [ ]:
# Create current team name filter 
justteam = ~df2022_merged.duplicated(subset=['Name'],keep='last')
# Create total stats filter
juststats = ~df2022_merged.duplicated(subset=['Name'],keep='first')


df2022_merged = (pd.merge(df2022_merged[juststats], df2022_merged[justteam][['Name','Tm']], 
                          left_on="Name", right_on="Name", how="left")
                            .rename(columns ={'Tm_x':'TeamOrTot', 'Tm_y':'Team'}) 
         )

df2022_merged['NameYear'] = df2022_merged['Name'].apply(strip_accents_and_punctuation).str.replace(' ','_') + '_2022' 
df2022_merged = df2022_merged.set_index('NameYear')

Minor bookkeeping:
Update a few Teams with correct info (only applies to 15 players, not that important):


mask = df2022_merged.CurrentTeam.notna() & (df2022_merged.CurrentTeam != df2022_merged.Team)
df2022_merged.loc[mask,'Team'] = df2022_merged.loc[mask,'CurrentTeam']

df2022_merged = df2022_merged.drop(columns=["CurrentTeam"])
df2022_merged = df2022_merged.rename(columns= {'CurrentSalary': 'Salary'})

df2022_merged.head(5)

## Save merged data

In [27]:
df2021_merged.to_csv(r'C:\Users\Vincenzo\Documents\Master 2. Semester\GutHub_Sofi\BigDataNBA\data\df2021_merged.csv')
df2022_merged.to_csv(r'C:\Users\Vincenzo\Documents\Master 2. Semester\GutHub_Sofi\BigDataNBA\data\df2022_merged.csv')

# (2) Clean salary data

## Load files

In [36]:
df2016_raw = pd.read_csv('./data/df2016_raw.csv')
df2017_raw = pd.read_csv('./data/df2017_raw.csv')
df2018_raw = pd.read_csv('./data/df2018_raw.csv')
df2019_raw = pd.read_csv('./data/df2019_raw.csv')
df2020_merged = pd.read_csv('./data/df2020_merged.csv')
df2021_merged = pd.read_csv('./data/df2021_merged.csv')

df  = pd.concat([df2016_raw, df2017_raw, df2018_raw, df2019_raw, df2020_merged, df2021_merged])
df = df.set_index('NameYear')

df2022 = pd.read_csv('./data/df2022_merged.csv')
df2022 = df2022.set_index('2023')
df2022 = df2022.drop("Player_2022")  #spurious entry

KeyError: "None of ['2023'] are in the columns"

## Delete unnecessary entries

We first employ a very light common-sense filter on our entries.  To train our player value model, we wish to keep only players who: 
1.  Played in at least one game the previous year
2.  Had a salary the next year (i.e. actually received a contract in free agency)

In [37]:
#  1. Mask for players who played in at least one game the previous year

df['G'] = df['G'].apply(lambda s: pd.to_numeric(s, errors='coerce'))
mask1 = df['G'].isna()
df = df[~mask1]

#  2. Mask for players who had a salary the next year

mask2  = (df['NextSal'] == '0') | df['NextSal'].isna()  
df = df[~mask2]
df.sample(3)

,2023,PrevYear,Name,Pos,Type,OTm,PrevStats,NTm,ID,Weight,...,AST%,BLK%,DRB%,FTr,ORB%,STL%,TOV%,TRB%,Salary,Team
NameYear,,,,,,,,,,,,,,,,,,,,,
Ryan_Arcidiacono_2020,NaN,2020.0,Ryan Arcidiacono,G,UFA,CHI,"3.1 Pts, 1.5 Reb, 1.3 Ast",BOS,/players/a/arcidry01.html,195.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Kelly_Oubre_Jr_2020,NaN,2020.0,Kelly Oubre Jr.,F,UFA,GSW,"15.4 Pts, 6.0 Reb, 1.3 Ast",CHO,/players/o/oubreke01.html,203.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Max_Strus_2020,NaN,2020.0,Max Strus,G,RFA,MIA,"6.1 Pts, 1.1 Reb, 0.6 Ast",MIA,/players/s/strusma01.html,215.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Clean salries and normalize by salary cap

Since we are looking at data for multiple years, it is important to normalize by salary cap for apples-to-apples comparisons in our model.  We chose the 2021 salary cap for simplicity.

In [38]:
def clean_salary(sal, year):
    ''' Adds all salaries earned per year into one salary (in millions of dollars)
        "< $Minimum" is treated as $0
    '''
    newlist = [0]
    for elem in sal.split('$'):
        try:
            elem = elem.replace('<','').replace(',','').replace('(TW)','').replace('Minimum','0')
            elem =  float(elem)
            newlist.append(elem)
        except:
            continue
            
    salary = sum(newlist)/1000000
    
    return salary

# Salary Cap Data taken from spotrac.com/nba/cba
salarycap = {2016:  94.143, 2017: 99.093,   2018: 101.869,  2019:  109.14,   
             2020:  109.14,    2021: 112.414,   2022: 123.655,  2023: 136.021}   

def clean_and_norm_salary(sal, year):
    ''' Adds all salaries earned per year into one salary (in millions of dollars)
        "< $Minimum" is treated as $0
        Then normalizes by salary cap (in 2021 salary cap dollars)
    '''
    newlist = [0]
    for elem in sal.split('$'):
        try:
            elem = elem.replace('<','').replace(',','').replace('Minimum','0')
            elem =  float(elem)
            newlist.append(elem)
        except:
            continue
            
    salary = sum(newlist)/1000000
    salary = salary*salarycap[2021]/salarycap[year]
        
    return salary

for index, row in df.iterrows():
    
    prevyear = df.loc[index, 'PrevYear']
    nextyear = df.loc[index, 'PrevYear'] + 1 

    prevsal = df.loc[index, 'PrevSal']
    nextsal = df.loc[index, 'NextSal']
    
    df.loc[index, 'PrevSalClean'] = clean_salary(prevsal, prevyear)
    df.loc[index, 'NextSalClean'] = clean_salary(nextsal, nextyear)
    df.loc[index, 'PrevSalNorm'] = clean_and_norm_salary(prevsal, prevyear)
    df.loc[index, 'NextSalNorm'] = clean_and_norm_salary(nextsal, nextyear)

cols = ['NextSal', 'NextSalClean','NextSalNorm']
df[cols].sample(8)

,NextSal,NextSalClean,NextSalNorm
NameYear,,,
JaVale_McGee_2020,"$5,000,000",5.000000,5.000000
Saben_Lee_2020,"$1,489,065",1.489065,1.489065
Juwan_Morgan_2020,"$19,186",0.019186,0.019186
Raul_Neto_2020,"$2,089,448",2.089448,2.089448
Derrick_Rose_2020,"$13,445,120",13.445120,13.445120
Gorgui_Dieng_2020,"$4,000,000",4.000000,4.000000
Otto_Porter_Jr_2020,"$2,389,641",2.389641,2.389641
PJ_Tucker_2020,"$7,000,000",7.000000,7.000000


In [40]:
for index, row in df2022.iterrows():
    
    salary = df2022.loc[index, 'CurrentSalary']
    try:
        df2022.loc[index, 'SalClean'] = clean_salary(CurrentSalary, 2023)
    except:
        df2022.loc[index, 'SalClean'] = np.nan
        
    try:
        df2022.loc[index, 'SalNorm'] = norm_salary(CurrentSalary, 2023)
    except:
        df2022.loc[index, 'SalNorm'] = np.nan

df2022.sample(3)

,Unnamed: 0,Name,Pos,Age,Tm,G,GS,MP,FG,FGA,...,TS%,USG%,VORP,WS,WS/48,CurrentSalary,CurrentTeam,ID,SalClean,SalNorm
461,461,Saben Lee,PG,22,DET,37,0,16.3,1.9,4.8,...,.499,17.3,0.2,0.9,.073,NaN,NaN,/players/l/leesa01.html,NaN,NaN
295,295,Maurice Harkless,SF,28,SAC,47,24,18.4,1.8,3.9,...,.561,11.2,-0.3,0.5,.026,NaN,NaN,NaN,NaN,NaN
105,105,Shaq Buchanan,SG,25,MEM,2,0,5.0,0.5,2.0,...,.250,24.6,0.0,0.0,-0.230,NaN,NaN,NaN,NaN,NaN


## Save cleaned data

df.to_csv(r'/Users/richardsihombing/Documents/BigDataNBA/data/cleaned_pastyears.csv')
df2021.to_csv(r'/Users/richardsihombing/Documents/BigDataNBA/data/cleaned_currentyear.csv')



In [41]:
df.to_csv(r'C:\Users\Vincenzo\Documents\Master 2. Semester\GutHub_Sofi\BigDataNBA\data\cleaned_pastyears.csv')
df2022.to_csv(r'C:\Users\Vincenzo\Documents\Master 2. Semester\GutHub_Sofi\BigDataNBA\data\cleaned_currentyear.csv')
